In [3]:
!pip install geopandas
!pip install GeoText
!pip install geopy

In [19]:
import matplotlib as plt
%matplotlib inline
import pandas as pd
import matplotlib.patches as mpatches
import plotly.express as px
from geotext import GeoText
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import requests
from bs4 import BeautifulSoup
import os
import re
from functools import reduce

In [5]:
x = '2020-03-04 Coronavirus Tweets.CSV'
y = 'geolocation_test.csv'

In [6]:
df2 = pd.read_csv(x,encoding='utf-8')
df2_text = df2[["text"]]
df2_text.to_string()
df2_text1 = df2_text.reset_index(drop=True)
df2_text1['geotext'] = df2_text1["text"].apply(lambda text: GeoText(text))

In [7]:
a = []
for i in range(0,len(df2_text1)):a  += (df2_text1['geotext'][i].cities)
city_mentions = pd.DataFrame(a, columns=['city'])

In [8]:
city_mentions

,city
0,Hyderabad
1,Wuhan
2,Wuhan
3,Mumbai
4,Brussels
...,...
1285,Florida
1286,March
1287,Vancouver
1288,Ontario


In [10]:
rank = city_mentions['city'].value_counts(ascending = False)
rank

Corona       245
Wuhan        122
March         60
Obama         59
Seattle       38
            ... 
AKi            1
Detroit        1
Gary           1
Millcreek      1
Waterloo       1
Name: city, Length: 281, dtype: int64

In [11]:
cities_to_remove = ['Corona', 'March', 'Of','Most','Obama','Un','Una','Bo','Best','Ho','Nice','University','Man','Police','Como']

In [13]:
city_mentions = city_mentions[~city_mentions['city'].isin(cities_to_remove)]
city_mentions

,city
0,Hyderabad
1,Wuhan
2,Wuhan
3,Mumbai
4,Brussels
...,...
1284,Barrington
1285,Florida
1287,Vancouver
1288,Ontario


In [15]:
rank = city_mentions['city'].value_counts()
rank

Wuhan        122
Seattle       38
Tokyo         29
Delhi         28
London        24
            ... 
AKi            1
Gary           1
Millcreek      1
Lancaster      1
Waterloo       1
Name: city, Length: 267, dtype: int64

In [17]:
rank = pd.DataFrame(rank)
rank

,city
Wuhan,122
Seattle,38
Tokyo,29
Delhi,28
London,24
...,...
AKi,1
Gary,1
Millcreek,1
Lancaster,1


In [ ]:
rank.to_csv(y)

# Geolocation

In [21]:
df = pd.read_csv("geolocation_test.csv",encoding='utf-8')
df

,Unnamed: 0,city
0,Wuhan,122
1,Seattle,38
2,Tokyo,29
3,Delhi,28
4,London,24
...,...,...
262,AKi,1
263,Gary,1
264,Millcreek,1
265,Lancaster,1


In [22]:
df.columns = ['city','count']

In [23]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [24]:
#use geopy's geolocator to find information on all the cities in the list
chrome_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36"

geolocator = Nominatim(timeout=10,user_agent=chrome_user_agent)

lat_lon = []
for i in df.city: 
    try:
        location = geolocator.geocode(i)
        if location:
            lat_lon.append(location)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%
             (i, e))

In [25]:
#create a df for raw output from the geolocator, maybe put cleaning directly after this; or append to original df
city_data = pd.DataFrame(lat_lon, columns=['raw_data','raw_data2'])

#re-order the cols
city_data = city_data[['raw_data2', 'raw_data']]

#get the city names from raw_data
city_data['city'] = city_data['raw_data'].str.split(',').str[0]

In [26]:
#change the coordinates to a string
city_data['raw_data2'] = city_data['raw_data2'].astype(str)

#split the coordinates using the comma as the delimiter
city_data[['lat','lon']] = city_data.raw_data2.str.split(",",expand=True,)

#remove the parentheses
city_data['lat'] = city_data['lat'].map(lambda x:x.lstrip('()'))
city_data['lon'] = city_data['lon'].map(lambda x:x.rstrip('()'))

#convert the lat and lon columns back to floats
city_data = city_data.astype({'lat': 'float64', 'lon': 'float64'})

#drop the raw data columns
city_data.drop(['raw_data2', 'raw_data'], axis = 1, inplace=True)

In [27]:
city_data

,city,lat,lon
0,武汉市,30.595105,114.299935
1,Seattle,47.603832,-122.330062
2,東京都,35.682839,139.759455
3,Delhi,28.651718,77.221939
4,London,51.507322,-0.127647
...,...,...,...
262,安芸国,34.429010,132.485385
263,Gary,41.602129,-87.337137
264,Millcreek,40.699891,-111.854093
265,Lancaster County,40.080670,-76.241128


In [29]:
#merge the two dataframes
merged = pd.merge(df, city_data, left_index=True, right_index=True)

In [30]:
merged

,city_x,count,city_y,lat,lon
0,Wuhan,122,武汉市,30.595105,114.299935
1,Seattle,38,Seattle,47.603832,-122.330062
2,Tokyo,29,東京都,35.682839,139.759455
3,Delhi,28,Delhi,28.651718,77.221939
4,London,24,London,51.507322,-0.127647
...,...,...,...,...,...
262,AKi,1,安芸国,34.429010,132.485385
263,Gary,1,Gary,41.602129,-87.337137
264,Millcreek,1,Millcreek,40.699891,-111.854093
265,Lancaster,1,Lancaster County,40.080670,-76.241128


In [31]:
merged.to_csv("geolocation_test.csv")